In [1]:
import os
import cv2
import PIL
import scipy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Definisikan path ke dataset FER-2013
dataset_dir = "dataset"

# Definisikan path ke folder train dan validasi
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

In [3]:
# Parameter pelatihan
batch_size = 32
epochs = 30
input_shape = (48, 48, 1)  # Sesuaikan dengan ukuran gambar dalam dataset

# Membangun model CNN untuk pengenalan ekspresi wajah
model = keras.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(7, activation='softmax')
])


# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Augmentasi data untuk meningkatkan variasi dataset
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data generator untuk training dan validasi
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(48, 48),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(test_dir,
                                                              target_size=(48, 48),
                                                              color_mode="grayscale",
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

# Melatih model dengan data generator
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=len(validation_generator))

# Simpan model setelah melatih
model.save('emotion_model.h5')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/30
898/898 [==============================] - 373s 414ms/step - loss: 1.7560 - accuracy: 0.2806 - val_loss: 1.6233 - val_accuracy: 0.3469
Epoch 2/30
898/898 [==============================] - 241s 269ms/step - loss: 1.6229 - accuracy: 0.3551 - val_loss: 1.4652 - val_accuracy: 0.4250
Epoch 3/30
898/898 [==============================] - 232s 259ms/step - loss: 1.4529 - accuracy: 0.4341 - val_loss: 1.3070 - val_accuracy: 0.4992
Epoch 4/30
898/898 [==============================] - 224s 249ms/step - loss: 1.3624 - accuracy: 0.4755 - val_loss: 1.2488 - val_accuracy: 0.5189
Epoch 5/30
898/898 [==============================] - 224s 249ms/step - loss: 1.2920 - accuracy: 0.5035 - val_loss: 1.1975 - val_accuracy: 0.5404
Epoch 6/30
898/898 [==============================] - 224s 250ms/step - loss: 1.2535 - accuracy: 0.5205 - val_loss: 1.1600 - val_accuracy: 0.5573
Epoch 7/30
898/898 [===================

d:\Berbagai Code 2\Python\myenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
from keras.models import load_model

emotion_model = load_model('emotion_model.h5')